<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#篩選-2020/1/1-還活著的公司，且上市時間大於7年公司，並去除銀行證券業" data-toc-modified-id="篩選-2020/1/1-還活著的公司，且上市時間大於7年公司，並去除銀行證券業-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>篩選 2020/1/1 還活著的公司，且上市時間大於7年公司，並去除銀行證券業</a></span></li><li><span><a href="#最近年度股東權益報酬率＞平均值(市場及產業)" data-toc-modified-id="最近年度股東權益報酬率＞平均值(市場及產業)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>最近年度股東權益報酬率＞平均值(市場及產業)</a></span></li><li><span><a href="#3年平均股東權益報酬率＞15%" data-toc-modified-id="3年平均股東權益報酬率＞15%-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3年平均股東權益報酬率＞15%</a></span></li><li><span><a href="#最近年度毛利率＞產業平均值" data-toc-modified-id="最近年度毛利率＞產業平均值-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>最近年度毛利率＞產業平均值</a></span></li><li><span><a href="#7年內市值增加值/7年內保留盈餘增加值&gt;1" data-toc-modified-id="7年內市值增加值/7年內保留盈餘增加值>1-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>7年內市值增加值/7年內保留盈餘增加值&gt;1</a></span></li><li><span><a href="#(最近年度自由現金流量/7年前自由現金流量)-1-&gt;-=-1" data-toc-modified-id="(最近年度自由現金流量/7年前自由現金流量)-1->-=-1-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>(最近年度自由現金流量/7年前自由現金流量)-1 &gt; = 1</a></span></li><li><span><a href="#目前市值/未來10年自由現金流量折現值&lt;1" data-toc-modified-id="目前市值/未來10年自由現金流量折現值<1-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>目前市值/未來10年自由現金流量折現值&lt;1</a></span></li><li><span><a href="#投組" data-toc-modified-id="投組-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>投組</a></span></li></ul></div>

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tejapi

In [73]:
tejapi.ApiConfig.api_key = 'Your Key'
tejapi.ApiConfig.ignoretz = True

# 篩選 2020/1/1 還活著的公司，且上市時間大於7年公司，並去除銀行證券業

- 交易目標，在 2020/1/1 找出投資標的，持有至 2020/12/31

In [74]:
comp_data = tejapi.get('TWN/AIND',
                   elist_day1 ={'lt':'2013-01-01'},
                   ind = {'ne':('28', '30')},
                   mkt = 'TSE',
                   opts = {'columns':['coid', 'elist_day1', 
                           'ind']},
                   chinese_column_name=True,
                   paginate=True
                  )

In [75]:
comp_list = list(comp_data['公司簡稱'])

In [76]:
comp_data

,公司簡稱,最近上市日,TEJ 產業別
None,,,
0,1101,1962-02-09,11
1,1102,1962-06-18,11
2,1103,1969-11-27,11
3,1104,1971-02-02,11
4,1108,1990-06-06,11
...,...,...,...
739,9943,2002-08-26,27
740,9944,2002-08-26,99
741,9945,1992-04-30,99


In [77]:
len(comp_list)

744

# 最近年度股東權益報酬率＞平均值(市場及產業)

 - 市場列表

In [78]:
industry_list = comp_data["TEJ 產業別"].unique().tolist()

In [79]:
len(industry_list)

19

- 製作字典

In [80]:
industry_data = tejapi.get('TWN/AIND',
                    ind = industry_list,
                    mkt = 'TSE',
                    opts = {'columns':['coid', 'ind']},      
                    chinese_column_name=True,
                    paginate=True
                   ).reset_index(drop=True)

In [81]:
industry_data

,公司簡稱,TEJ 產業別
0,1101,11
1,1102,11
2,1103,11
3,1104,11
4,1108,11
...,...,...
925,9944,99
926,9945,99
927,9946,25
928,9955,99


- 市場字典

In [82]:
industry_dict = {}
for i in industry_list:
    industry_dict[i] = industry_data[industry_data['TEJ 產業別'] == i]['公司簡稱'].tolist()

In [83]:
industry_dict['13']

['1301',
 '1303',
 '1304',
 '1305',
 '1307',
 '1308',
 '1309',
 '1310',
 '1312',
 '1313',
 '1314',
 '1315',
 '1321',
 '1323',
 '1324',
 '1325',
 '1326',
 '1337',
 '1340',
 '1341',
 '4306']

- 市場ROE

In [84]:
industry_roe = {}
for i in industry_list:
    data = tejapi.get('TWN/ACRQMTAB',
                  coid= industry_dict[i],
                  mdate={'gt':'2019-01-01','lt':'2019-12-31'},
                  opts={'columns':['coid', 'mdate', 'roe']},
                  paginate=True,
                  chinese_column_name=True,
                 ).reset_index(drop=True)
    industry_roe[i] = data.groupby('公司').mean().mean()[0]

In [85]:
industry_roe

{'11': 4.015000000000001,
 '12': 5.813863636363635,
 '13': 3.337619047619047,
 '25': 2.8016509433962264,
 '22': 4.647714285714287,
 '14': -1.293214285714286,
 '99': 5.979601449275362,
 '29': 3.2005952380952385,
 '23': 3.082491909385112,
 '15': 3.337287234042554,
 '16': 3.345166666666666,
 '17': 3.3955555555555548,
 '18': -0.6959999999999997,
 '19': 1.24875,
 '20': 1.5639516129032256,
 '21': 4.847272727272728,
 '26': 1.5829347826086957,
 '97': 6.1740625,
 '27': 4.719411764705882}

- 個股ROE

In [86]:
ROE_data = tejapi.get('TWN/ACRQMTAB',
                      coid=comp_list,
                      mdate={'gt':'2019-01-01','lt':'2019-12-31'},
                      opts={'columns':['coid', 'mdate', 'roe']},
                      paginate=True,
                      chinese_column_name=True,
                     ).set_index('年/月')

In [87]:
ROE_data = ROE_data.groupby('公司').resample('Y').mean().reset_index()

In [88]:
ROE_data.rename(columns={'公司':'公司簡稱'}, inplace=True)

In [89]:
ROE_data = ROE_data.merge(comp_data, on='公司簡稱')
ROE_data

,公司簡稱,年/月,淨值報酬%,最近上市日,TEJ 產業別
0,1101,2019-12-31,7.5550,1962-02-09,11
1,1102,2019-12-31,7.7000,1962-06-18,11
2,1103,2019-12-31,-0.8300,1969-11-27,11
3,1104,2019-12-31,3.4325,1971-02-02,11
4,1108,2019-12-31,-0.0750,1990-06-06,11
...,...,...,...,...,...
722,9943,2019-12-31,13.4725,2002-08-26,27
723,9944,2019-12-31,0.8825,2002-08-26,99
724,9945,2019-12-31,12.4125,1992-04-30,99
725,9955,2019-12-31,1.1475,2008-01-21,99


- 將產業別替換成產業平均

In [90]:
ROE_data['產業淨值報酬%'] = ROE_data['TEJ 產業別'].apply(lambda x: industry_roe[x])

In [91]:
ROE_data

,公司簡稱,年/月,淨值報酬%,最近上市日,TEJ 產業別,產業淨值報酬%
0,1101,2019-12-31,7.5550,1962-02-09,11,4.015000
1,1102,2019-12-31,7.7000,1962-06-18,11,4.015000
2,1103,2019-12-31,-0.8300,1969-11-27,11,4.015000
3,1104,2019-12-31,3.4325,1971-02-02,11,4.015000
4,1108,2019-12-31,-0.0750,1990-06-06,11,4.015000
...,...,...,...,...,...,...
722,9943,2019-12-31,13.4725,2002-08-26,27,4.719412
723,9944,2019-12-31,0.8825,2002-08-26,99,5.979601
724,9945,2019-12-31,12.4125,1992-04-30,99,5.979601
725,9955,2019-12-31,1.1475,2008-01-21,99,5.979601


In [92]:
set_1 = set(ROE_data[ROE_data['淨值報酬%'] > ROE_data['產業淨值報酬%']]['公司簡稱'])

In [93]:
len(set_1)

362

# 3年平均股東權益報酬率＞15%

In [94]:
ROE_data_3Y = tejapi.get('TWN/ACRQMTAB',
                      coid=comp_list,
                      mdate={'gt':'2017-01-01','lt':'2019-12-31'},
                      opts={'columns':['coid', 'mdate', 'roe']},
                      paginate=True,
                      chinese_column_name=True,
                     ).set_index('年/月')

In [95]:
# 獲得年度ROE
ROE_data_3Y = ROE_data_3Y.groupby('公司').resample('Y').mean()
ROE_data_3Y = ROE_data_3Y.groupby('公司').mean()
ROE_data_3Y

,淨值報酬%
公司,
1101,6.466667
1102,5.365833
1103,-0.708333
1104,3.851667
1108,-0.590833
...,...
9943,13.406667
9944,1.159167
9945,12.715000


In [96]:
set_2 = set(ROE_data_3Y[ROE_data_3Y['淨值報酬%'] >= 15].index)

In [97]:
len(set_2)

18

# 最近年度毛利率＞產業平均值

In [98]:
data_gm = tejapi.get('TWN/AIM1A',
                  coid= comp_list,
                  mdate={'gt':'2019-01-01','lt':'2019-12-31'},
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R105']},
                  paginate=True,
                  chinese_column_name=True,
                 ).set_index('財報年月')
data_gm

,公司代碼,營業毛利率
財報年月,,
2019-03-01,1101,25.59
2019-06-01,1101,27.87
2019-09-01,1101,29.16
2019-12-01,1101,29.25
2019-03-01,1102,23.67
...,...,...
2019-12-01,9955,3.72
2019-03-01,9958,16.03
2019-06-01,9958,14.49


In [99]:
# 獲得年度毛利率

data_gm_Y = data_gm.groupby('公司代碼').resample('Y').mean()
data_gm_Y

,,營業毛利率
公司代碼,財報年月,
1101,2019-12-31,27.9675
1102,2019-12-31,26.2750
1103,2019-12-31,12.5075
1104,2019-12-31,10.1700
1108,2019-12-31,5.9475
...,...,...
9943,2019-12-31,57.5750
9944,2019-12-31,24.0450
9945,2019-12-31,21.6950


In [100]:
# 產業毛利率
industry_gm = {}
for i in industry_dict:
    data_gm = tejapi.get('TWN/AIM1A',
                  coid= industry_dict[i],
                  mdate={'gt':'2019-01-01','lt':'2019-12-31'},
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R105']},
                  paginate=True,
                  chinese_column_name=True,
                 ).reset_index(drop=True)

    industry_gm[i] = data_gm.groupby('公司代碼').mean().mean()[0]

In [101]:
industry_gm

{'11': 17.178214285714287,
 '12': 25.807083333333328,
 '13': 9.723095238095235,
 '25': 15.859103773584906,
 '22': 21.793263888888895,
 '14': 10.78892857142857,
 '99': 28.77957792207792,
 '29': 41.89625,
 '23': 20.592081339712912,
 '15': 23.206562500000004,
 '16': 13.129500000000002,
 '17': 32.08277777777778,
 '18': 19.3315,
 '19': 16.1225,
 '20': 7.899036458333335,
 '21': 22.036590909090908,
 '26': 17.66695652173913,
 '97': 14.72625,
 '27': 41.76264705882353}

In [102]:
data_gm_Y = data_gm_Y.reset_index().rename(columns={'公司代碼':'公司簡稱'})
data_gm_Y = data_gm_Y.merge(comp_data, on='公司簡稱')
data_gm_Y


,公司簡稱,財報年月,營業毛利率,最近上市日,TEJ 產業別
0,1101,2019-12-31,27.9675,1962-02-09,11
1,1102,2019-12-31,26.2750,1962-06-18,11
2,1103,2019-12-31,12.5075,1969-11-27,11
3,1104,2019-12-31,10.1700,1971-02-02,11
4,1108,2019-12-31,5.9475,1990-06-06,11
...,...,...,...,...,...
739,9943,2019-12-31,57.5750,2002-08-26,27
740,9944,2019-12-31,24.0450,2002-08-26,99
741,9945,2019-12-31,21.6950,1992-04-30,99
742,9955,2019-12-31,4.7925,2008-01-21,99


In [103]:
data_gm_Y['產業營業毛利率'] = data_gm_Y['TEJ 產業別'].apply(lambda x: industry_gm[x])

In [104]:
data_gm_Y

,公司簡稱,財報年月,營業毛利率,最近上市日,TEJ 產業別,產業營業毛利率
0,1101,2019-12-31,27.9675,1962-02-09,11,17.178214
1,1102,2019-12-31,26.2750,1962-06-18,11,17.178214
2,1103,2019-12-31,12.5075,1969-11-27,11,17.178214
3,1104,2019-12-31,10.1700,1971-02-02,11,17.178214
4,1108,2019-12-31,5.9475,1990-06-06,11,17.178214
...,...,...,...,...,...,...
739,9943,2019-12-31,57.5750,2002-08-26,27,41.762647
740,9944,2019-12-31,24.0450,2002-08-26,99,28.779578
741,9945,2019-12-31,21.6950,1992-04-30,99,28.779578
742,9955,2019-12-31,4.7925,2008-01-21,99,28.779578


In [105]:
set_3 = set(data_gm_Y[data_gm_Y['營業毛利率'] > data_gm_Y['產業營業毛利率']]['公司簡稱'])

In [106]:
len(set_3)

333

# 7年內市值增加值/7年內保留盈餘增加值>1

In [107]:
MV_data = tejapi.get('TWN/AAPRCM1',
                coid = comp_list,
                mdate= ['2019/12/01','2013/01/01'],
                opts = {'columns':['coid', 'mdate', 'MV']},
                chinese_column_name=True,
                paginate=True
               ).reset_index(drop=True)
MV_data

,證券代碼,年月,市值(百萬元)
0,1101,2013-01-01,147318
1,1101,2019-12-01,238848
2,1102,2013-01-01,121321
3,1102,2019-12-01,161181
4,1103,2013-01-01,10798
...,...,...,...
1482,9945,2019-12-01,45347
1483,9955,2013-01-01,2941
1484,9955,2019-12-01,1951
1485,9958,2013-01-01,1835


In [108]:
MV_data = MV_data.pivot_table(index='證券代碼', columns='年月').reset_index()

In [109]:
MV_data['市值增加值(百萬元)'] = MV_data.iloc[:,2] - MV_data.iloc[:,1]
MV_data

證券代碼             市值(百萬元)                     市值增加值(百萬元)
年月        2013-01-01 00:00:00 2019-12-01 00:00:00           
0    1101            147318.0            238848.0    91530.0
1    1102            121321.0            161181.0    39860.0
2    1103             10798.0             17316.0     6518.0
3    1104             10266.0             12811.0     2545.0
4    1108              2732.0              3392.0      660.0
..    ...                 ...                 ...        ...
739  9943              5930.0             10504.0     4574.0
740  9944              1813.0              1898.0       85.0
741  9945             66188.0             45347.0   -20841.0
742  9955              2941.0              1951.0     -990.0
743  9958              1835.0             15147.0    13312.0

[744 rows x 4 columns]

In [110]:
MV_data.columns = MV_data.columns.droplevel(1)

In [111]:
MV_data

,證券代碼,市值(百萬元),市值(百萬元),市值增加值(百萬元)
0,1101,147318.0,238848.0,91530.0
1,1102,121321.0,161181.0,39860.0
2,1103,10798.0,17316.0,6518.0
3,1104,10266.0,12811.0,2545.0
4,1108,2732.0,3392.0,660.0
...,...,...,...,...
739,9943,5930.0,10504.0,4574.0
740,9944,1813.0,1898.0,85.0
741,9945,66188.0,45347.0,-20841.0
742,9955,2941.0,1951.0,-990.0


In [112]:
RE_data = tejapi.get('TWN/EWIFINQ',
                coid = comp_list,
                mdate= ['2019/12/01','2013/03/01'],
                opts = {'columns':['coid', 'mdate', 'ac_2341']},
                chinese_column_name=True,
                paginate=True
               ).reset_index(drop=True)
RE_data

,證券碼,財務資料日,保留盈餘
0,1101,2013-03-01,45521147.0
1,1101,2019-12-01,65626033.0
2,1102,2019-12-01,108564355.0
3,1102,2013-03-01,88960768.0
4,1103,2013-03-01,8044970.0
...,...,...,...
1457,9945,2019-12-01,39901502.0
1458,9955,2019-12-01,-261304.0
1459,9955,2013-03-01,224981.0
1460,9958,2019-12-01,1086259.0


In [113]:
RE_data = RE_data.pivot_table(index='證券碼', columns='財務資料日').reset_index()

In [114]:
RE_data['保留盈餘增加值(仟元)'] = RE_data.iloc[:,2] - RE_data.iloc[:,1]

In [115]:
RE_data

證券碼                保留盈餘                     保留盈餘增加值(仟元)
財務資料日       2013-03-01 00:00:00 2019-12-01 00:00:00            
0      1101          45521147.0          65626033.0  20104886.0
1      1102          88960768.0         108564355.0  19603587.0
2      1103           8044970.0          10660775.0   2615805.0
3      1104           6765931.0          11013644.0   4247713.0
4      1108            199978.0            269466.0     69488.0
..      ...                 ...                 ...         ...
726    9943           1984164.0           2724144.0    739980.0
727    9944           1347554.0           1456085.0    108531.0
728    9945           8729008.0          39901502.0  31172494.0
729    9955            224981.0           -261304.0   -486285.0
730    9958            397410.0           1086259.0    688849.0

[731 rows x 4 columns]

In [116]:
RE_data.columns = RE_data.columns.droplevel(1)

In [117]:
RE_data = RE_data.rename(columns={'證券碼':'證券代碼'})

In [118]:
RE_data

,證券代碼,保留盈餘,保留盈餘,保留盈餘增加值(仟元)
0,1101,45521147.0,65626033.0,20104886.0
1,1102,88960768.0,108564355.0,19603587.0
2,1103,8044970.0,10660775.0,2615805.0
3,1104,6765931.0,11013644.0,4247713.0
4,1108,199978.0,269466.0,69488.0
...,...,...,...,...
726,9943,1984164.0,2724144.0,739980.0
727,9944,1347554.0,1456085.0,108531.0
728,9945,8729008.0,39901502.0,31172494.0
729,9955,224981.0,-261304.0,-486285.0


In [119]:
RE_MV_data = RE_data.merge(MV_data, on='證券代碼')

In [120]:
RE_MV_data

,證券代碼,保留盈餘,保留盈餘,保留盈餘增加值(仟元),市值(百萬元),市值(百萬元),市值增加值(百萬元)
0,1101,45521147.0,65626033.0,20104886.0,147318.0,238848.0,91530.0
1,1102,88960768.0,108564355.0,19603587.0,121321.0,161181.0,39860.0
2,1103,8044970.0,10660775.0,2615805.0,10798.0,17316.0,6518.0
3,1104,6765931.0,11013644.0,4247713.0,10266.0,12811.0,2545.0
4,1108,199978.0,269466.0,69488.0,2732.0,3392.0,660.0
...,...,...,...,...,...,...,...
726,9943,1984164.0,2724144.0,739980.0,5930.0,10504.0,4574.0
727,9944,1347554.0,1456085.0,108531.0,1813.0,1898.0,85.0
728,9945,8729008.0,39901502.0,31172494.0,66188.0,45347.0,-20841.0
729,9955,224981.0,-261304.0,-486285.0,2941.0,1951.0,-990.0


In [121]:
RE_MV_data['指標'] = RE_MV_data['市值增加值(百萬元)']*1000 / RE_MV_data['保留盈餘增加值(仟元)']

In [122]:
RE_MV_data

,證券代碼,保留盈餘,保留盈餘,保留盈餘增加值(仟元),市值(百萬元),市值(百萬元),市值增加值(百萬元),指標
0,1101,45521147.0,65626033.0,20104886.0,147318.0,238848.0,91530.0,4.552625
1,1102,88960768.0,108564355.0,19603587.0,121321.0,161181.0,39860.0,2.033301
2,1103,8044970.0,10660775.0,2615805.0,10798.0,17316.0,6518.0,2.491776
3,1104,6765931.0,11013644.0,4247713.0,10266.0,12811.0,2545.0,0.599146
4,1108,199978.0,269466.0,69488.0,2732.0,3392.0,660.0,9.498043
...,...,...,...,...,...,...,...,...
726,9943,1984164.0,2724144.0,739980.0,5930.0,10504.0,4574.0,6.181248
727,9944,1347554.0,1456085.0,108531.0,1813.0,1898.0,85.0,0.783186
728,9945,8729008.0,39901502.0,31172494.0,66188.0,45347.0,-20841.0,-0.668570
729,9955,224981.0,-261304.0,-486285.0,2941.0,1951.0,-990.0,2.035843


In [123]:
set_4 = set(RE_MV_data[RE_MV_data['指標']>1]['證券代碼'])

# (最近年度自由現金流量/7年前自由現金流量)-1 > = 1

In [124]:
cash_data = tejapi.get('TWN/AIM1A',
                  coid= comp_list,
                  mdate={'gt':'2013-01-01','lt':'2019-12-31'},
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R69B']},
                  paginate=True,
                  chinese_column_name=True,
                 ).set_index('財報年月')

In [125]:
cash_data

,公司代碼,自由現金流量(D)
財報年月,,
2013-03-01,1101,5306821.0
2013-06-01,1101,12865033.0
2013-09-01,1101,24103144.0
2013-12-01,1101,15642459.0
2014-03-01,1101,3183392.0
...,...,...
2018-12-01,9958,-3002460.0
2019-03-01,9958,424515.0
2019-06-01,9958,726185.0


In [126]:
cash_data_Y = cash_data.groupby('公司代碼').resample('Y').sum()
cash_data_Y

自由現金流量(D)
公司代碼 財報年月                  
1101 2013-12-31  57917457.0
     2014-12-31  53207415.0
     2015-12-31  58352078.0
     2016-12-31  54452838.0
     2017-12-31  42720876.0
...                     ...
9958 2015-12-31   2158130.0
     2016-12-31    972365.0
     2017-12-31  -2647614.0
     2018-12-31  -4320923.0
     2019-12-31   4829041.0

[5208 rows x 1 columns]

In [127]:
cash_data_Y = cash_data_Y.pivot_table(index='公司代碼', columns='財報年月')

In [128]:
cash_data_Y

自由現金流量(D)                                                              \
財報年月  2013-12-31  2014-12-31  2015-12-31  2016-12-31  2017-12-31  2018-12-31   
公司代碼                                                                           
1101  57917457.0  53207415.0  58352078.0  54452838.0  42720876.0  26852725.0   
1102  46947133.0  41345800.0  63372185.0   6123382.0  50548822.0  -1660422.0   
1103   4212133.0  11168837.0    932251.0   6568497.0   4755239.0  -1986660.0   
1104   2439127.0   4277015.0   1665891.0   3652217.0   2917845.0   2896415.0   
1108    379958.0   1533123.0   1454288.0   3520453.0   -731468.0   -784815.0   
...          ...         ...         ...         ...         ...         ...   
9943   1478047.0   1620839.0   2513008.0   2072953.0   1809563.0   2137331.0   
9944    659445.0  -1260591.0    196802.0    601199.0    634407.0    219764.0   
9945  41746981.0  25689050.0  12154418.0  31309567.0   9479063.0  22808830.0   
9955  -1464512.0  -1118727.0     86547.0    145189.0    320719.0    219023.0   
9958   -243161.0  -3700507.0   2158130.0    972365.0  -2647614.0  -4320923.0   

                  
財報年月  2019-12-31  
公司代碼              
1101  61416120.0  
1102  51474818.0  
1103   1359679.0  
1104   2659476.0  
1108    506610.0  
...          ...  
9943   4073848.0  
9944   -664155.0  
9945  50454160.0  
9955    154980.0  
9958   4829041.0  

[744 rows x 7 columns]

In [129]:
cash_data_Y.columns = cash_data_Y.columns.droplevel(0)

In [130]:
cash_data_Y['指標'] = (cash_data_Y['2019-12-31'] / cash_data_Y['2013-12-31']) - 1

In [131]:
cash_data_Y

財報年月,2013-12-31 00:00:00,2014-12-31 00:00:00,2015-12-31 00:00:00,2016-12-31 00:00:00,2017-12-31 00:00:00,2018-12-31 00:00:00,2019-12-31 00:00:00,指標
公司代碼,,,,,,,,
1101,57917457.0,53207415.0,58352078.0,54452838.0,42720876.0,26852725.0,61416120.0,0.060408
1102,46947133.0,41345800.0,63372185.0,6123382.0,50548822.0,-1660422.0,51474818.0,0.096442
1103,4212133.0,11168837.0,932251.0,6568497.0,4755239.0,-1986660.0,1359679.0,-0.677199
1104,2439127.0,4277015.0,1665891.0,3652217.0,2917845.0,2896415.0,2659476.0,0.090339
1108,379958.0,1533123.0,1454288.0,3520453.0,-731468.0,-784815.0,506610.0,0.333332
...,...,...,...,...,...,...,...,...
9943,1478047.0,1620839.0,2513008.0,2072953.0,1809563.0,2137331.0,4073848.0,1.756237
9944,659445.0,-1260591.0,196802.0,601199.0,634407.0,219764.0,-664155.0,-2.007142
9945,41746981.0,25689050.0,12154418.0,31309567.0,9479063.0,22808830.0,50454160.0,0.208570


In [132]:
set_5 = set(cash_data_Y[cash_data_Y['指標'] > 1].index)

In [133]:
len(set_5)

185

# 目前市值/未來10年自由現金流量折現值<1

In [134]:
import numpy_financial as npf

In [135]:
cash_data = tejapi.get('TWN/AIM1A',
                  coid= comp_list,
                  mdate={'gt':'2019-01-01','lt':'2019-12-31'},
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R69B']},
                  paginate=True,
                  chinese_column_name=True,
                 ).set_index('財報年月')

In [136]:
cash_data

,公司代碼,自由現金流量(D)
財報年月,,
2019-03-01,1101,6593029.0
2019-06-01,1101,15427641.0
2019-09-01,1101,20538786.0
2019-12-01,1101,18856664.0
2019-03-01,1102,2757925.0
...,...,...
2019-12-01,9955,65183.0
2019-03-01,9958,424515.0
2019-06-01,9958,726185.0


In [137]:
cash_data_Y = cash_data.groupby('公司代碼').resample('Y').sum()
cash_data_Y

,,自由現金流量(D)
公司代碼,財報年月,
1101,2019-12-31,61416120.0
1102,2019-12-31,51474818.0
1103,2019-12-31,1359679.0
1104,2019-12-31,2659476.0
1108,2019-12-31,506610.0
...,...,...
9943,2019-12-31,4073848.0
9944,2019-12-31,-664155.0
9945,2019-12-31,50454160.0


In [138]:
MV_data = tejapi.get('TWN/AAPRCM1',
                coid = comp_list,
                mdate= '2019/12/01',
                opts = {'columns':['coid', 'MV']},
                chinese_column_name=True,
                paginate=True
               ).set_index('證券代碼')
MV_data

,市值(百萬元)
證券代碼,
1101,238848
1102,161181
1103,17316
1104,12811
1108,3392
...,...
9943,10504
9944,1898
9945,45347


In [139]:
def Cashflows(y0):
    '''
    y0 為2019現金流量，單位千元
    '''
    y1 = y0 * 1000 * 1.15
    y2 = y1 * 1.15
    y3 = y2 * 1.05
    y4 = y3 * 1.05
    y5 = y4 * 1.05
    y6 = y5 * 1.05
    y7 = y6 * 1.05
    y8 = y7 * 1.05
    y9 = y8 * 1.05
    y10 = y9 * 1.05
    
    cashflows = np.array([y1,y2,y3,y4,y5,y6,y7,y8,y9,y10])
    
    return cashflows

In [140]:
set_6 = set()
for i in comp_list:
    try:
        y0 = cash_data_Y.loc[i]['自由現金流量(D)'][0]*1000
        
        cashflows = Cashflows(y0)
        
        earns = npf.npv(0.09, cashflows)

        mv = MV_data.loc[i]['市值(百萬元)'] * 1000000
        
        if (mv/earns) < 1 and (mv/earns) > 0:
            set_6.add(i)
    except:
        None

<ipython-input-140-1da6878a67c6>:12: RuntimeWarning: divide by zero encountered in true_divide
  if (mv/earns) < 1 and (mv/earns) > 0:


In [141]:
len(set_6)

581

# 投組

In [142]:
set_1 & set_2 & set_3 & set_4 & set_5 & set_6

{'2231', '2327', '2492', '3008', '5269'}